# VAR Model - Predicting Housing Price

This notebook demonstrates how I developed the VAR model for housing price prediction and made a price forecast for 12 months. Please read my data cleaning notebook for data cleaning, descriptive statistics, and EDA. 

Contents of this notebook: 
1. Granger Causality Test and Dickey-Fuller Test 

2. Modeling (Grid Search)

3. Model Validation 

3. Forecasting

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from statsmodels.tsa.api import VAR
import statsmodels.api as sm

from matplotlib.pylab import rcParams
import itertools

import warnings
warnings.filterwarnings('ignore')

First, I read a housing market dataset. USPS's population inflow data has alredy been merged to this dataset. For detailed data cleaning process and EDA, please read my data cleaning notebook. 

In [53]:
df = pd.read_csv('Data/df_realtor.csv')

In [54]:
print('Realtor housing price data covers from', df.yearmonth.min(), 'to' , df.yearmonth.max())
print('The data includes', df.zips.nunique(), 'zipcodes.' )

Realtor housing price data covers from 201607 to 202204
The data includes 189 zipcodes.


For the time series analysis,  I change the date to the datetime format and set date and zip code in indexes.

In [55]:
# change zips to 5 digit strings
df['zipcode'] = '000' + df.zips.astype('str')
df['zipcode'] = df.zipcode.str[-5:]

# change 'date' to the datatime
df['date']=pd.to_datetime(df.yearmonth, format='%Y%m')

# Set multiindex with date and zip code
df = df.set_index(['zipcode', 'date']).sort_index() 


I double-check that this data frame incldues only zip codes in Washington DC metropolitan area. 

In [56]:
# Zip codes included in Washington DC metropolitan area 
WashingtonDC = df[df['czname']=='Washington DC']

WashingtonDC.index.get_level_values(0).unique()


Index(['20001', '20002', '20003', '20005', '20007', '20008', '20009', '20010',
       '20011', '20012',
       ...
       '22307', '22308', '22309', '22310', '22312', '22314', '22315', '22630',
       '22642', '22712'],
      dtype='object', name='zipcode', length=189)

Drop unecessary columns. The model use three time series; listings price, number of listings, and population net inflow.

In [57]:
DC = WashingtonDC[['median_listing_price', 'active_listing_count', 'netinflow']].dropna()

DC

median_listing_price  active_listing_count  netinflow
zipcode date                                                             
20001   2018-01-01              730000.0                  81.0      -61.0
        2018-02-01              779000.0                  89.0     -138.0
        2018-03-01              729000.0                 103.0     -163.0
        2018-04-01              717000.0                 117.0      -94.0
        2018-05-01              754950.0                 125.0     -131.0
...                                  ...                   ...        ...
22712   2021-12-01              544750.0                   8.0       -8.0
        2022-01-01              475000.0                   6.0       12.0
        2022-02-01              450000.0                   7.0      -22.0
        2022-03-01              489950.0                   4.0       -4.0
        2022-04-01              465000.0                   5.0       -6.0

[9828 rows x 3 columns]

Before start analysis, I split my data set into train and test set. I use the 2021-12 as a cutoff month. As a consequence, train set has 48 observation and test set has 4 observations. 

In [58]:
# use 2021-09 as the cutoff point 
train_data = DC.iloc[DC.index.get_level_values('date') <= '2021-12']

test_data = DC.iloc[DC.index.get_level_values('date') > '2021-12'] 


In [59]:
print('Number of observation in train set for each zip code:', len(train_data.index.unique(level='date')))
print('Number of observation in test set for each zip code:', len(test_data.index.unique(level='date')))

Number of observation in train set for each zip code: 48
Number of observation in test set for each zip code: 4


# 1. Granger Causality Test and Dickey Fuller Test 

### Granger’s Causality Test
First, I run the Granger causality test for three time series in my model. I check the causality between all possible combinations of three time series. If the P-Values are smaller than 0.05, I drop the corresponding zip code from my sample. 
Below, I created a function for granger causality test. 


In [60]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """This function returns p-values of Granger causality test of all combinations of the time series.
    The rows are the response variable, columns are predictors 
    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

     

I run the granger causality check for all zip codes and drop the zip codes which failed the test.  

In [62]:
# To store zip codes which failed the test 
drop_list=[]

# list of zip codes 
zips=train_data.index.get_level_values(0).unique()

for x in zips:
    # Check granger causality and store in the data frame 
    g = pd.DataFrame(grangers_causation_matrix(train_data.loc[(x, ),], variables = train_data.columns))
    
    # Check each p-value in the stored table. If one of the p-values are greater than 0.05, add the zip code in drop list. 
    if max(g.iloc[1,0], g.iloc[2,0], g.iloc[0,1], g.iloc[2,1], g.iloc[0,2], g.iloc[1,2]) > 0.1: #0.1
        drop_list.append(x)
        
print(f'The following {len(drop_list)} zip codes failed the Granger Causality test:',  drop_list)



The following 32 zip codes failed the Granger Causality test: ['20017', '20019', '20110', '20115', '20165', '20170', '20171', '20187', '20194', '20637', '20646', '20664', '20685', '20706', '20736', '20743', '20832', '20852', '20855', '20871', '20872', '20874', '20895', '21701', '21702', '21703', '21754', '22041', '22182', '22202', '22630', '22712']


I drop the above zip codes. 

In [63]:
train_df = train_data.drop(drop_list)
test_df = test_data.drop(drop_list)
all_df = DC.drop(drop_list)


### Dickey-Fuller Test

I performed the Dickey-Fuller test for stationarity check of a time series. 
Below, I conducted the ADFuller test three times. The first test was with an original time series (i.e., no differencing, no moving average). And if a zip code fails this first test, I take a difference of a time series and redo the Dickey-Fuller test. If failed again, I take the second difference and try the ADFuller test. And any zip codes which failed the test with a second difference were dropped from this analysis. 

To run the test throughout all zip codes in my sample, first, I run the test and store the result in the data frame. And then, I evaluate the p-value and filter the zip code that failed the test. Zip codes that passed the first test are stored in 'diff0' list. Zip codes which need a second test with difference terms, I stored in 'diff1'.

In [64]:
# For each time series I tun the Dickey-Fuller test and report the p-values

# to store the results 
dtest = pd.DataFrame()
df_p = [] 
df_l = []
df_m = []
Zipcode = []

zips = train_df.index.unique(level='zipcode')
for x in zips:
    p_val_1 = adfuller(train_df.loc[(x, ),]['median_listing_price'])[1] # extract p-value
    p_val_2 = adfuller(train_df.loc[(x, ),]['active_listing_count'])[1] 
    p_val_3 = adfuller(train_df.loc[(x, ),]['netinflow'])[1] 
    df_p.append(p_val_1)
    df_l.append(p_val_2)
    df_m.append(p_val_3)
    Zipcode.append(x) 
    
dtest['Zipcode'] = Zipcode
dtest['Dickey_Fuller_p'] = df_p
dtest['Dickey_Fuller_l'] = df_l
dtest['Dickey_Fuller_m'] = df_m

dtest.head()

,Zipcode,Dickey_Fuller_p,Dickey_Fuller_l,Dickey_Fuller_m
0,20001,0.084303,0.420771,0.035095
1,20002,0.015976,0.994876,0.111127
2,20003,0.248776,0.000284,0.061371
3,20005,0.115617,0.137064,0.142104
4,20007,0.111817,0.002108,0.008003


The cell below filters out the zip codes by p-value. If it's smaller than 0.05, the zipcode is stored in 'diff0'. If it's larger than 0.05, the zipcode is sotred in 'diff1'. 

In [65]:
# Create a new column of maximum value of three Dickey_fuller p-values. Use this column to decide which zipcode need to take a difference.  
dtest['max_p'] = dtest[['Dickey_Fuller_p','Dickey_Fuller_l','Dickey_Fuller_m']].max(axis=1)

#List of zipcode which pass Dickey-Fuller test without taking difference 
diff0 = list(dtest[dtest.max_p <=0.05].Zipcode)

# List a zipcode which failed Dickey-Fuller test, thus need to take difference
diff1 = list(dtest[dtest.max_p > 0.05].Zipcode)


print( f'{ len(diff0)} zipcodes passed DF test in original scale.')
print( f'{ len(diff1)} zipcodes failed, so I take a difference and try again.')



8 zipcodes passed DF test in original scale.
149 zipcodes failed, so I take a difference and try again.


I take a difference and redo Dickey-fuller test.  

In [66]:
# for each series check Dickey-Fuller test and report the p-values

# to store the test results 
dtest1 = pd.DataFrame()
df_p = [] 
df_l = []
df_m = []
Zipcode = []

# now I do test only for the counties which failed the earlier test 
zips = diff1
for x in zips:
    p_val_1 = adfuller(train_df.diff().dropna().loc[(x, ),]['median_listing_price'])[1] # extract p-value
    p_val_2 = adfuller(train_df.diff().dropna().loc[(x, ),]['active_listing_count'])[1] 
    p_val_3 = adfuller(train_df.diff().dropna().loc[(x, ),]['netinflow'])[1] 
    df_p.append(p_val_1)
    df_l.append(p_val_2)
    df_m.append(p_val_3)
    Zipcode.append(x) 
    
dtest1['Zipcode'] = Zipcode
dtest1['Dickey_Fuller_p'] = df_p
dtest1['Dickey_Fuller_l'] = df_l
dtest1['Dickey_Fuller_m'] = df_m

dtest1.head()
    

,Zipcode,Dickey_Fuller_p,Dickey_Fuller_l,Dickey_Fuller_m
0,20001,2.298098e-13,2.359540e-03,1.378247e-13
1,20002,1.157793e-03,3.016470e-02,5.665424e-15
2,20003,4.445139e-06,1.147917e-03,4.224673e-15
3,20005,6.342952e-12,7.440427e-12,7.408380e-14
4,20007,3.928826e-08,1.260332e-03,2.248241e-03


In below, I filter the zip codes which failed the second DF test. 

In [67]:
# Create a new column of maximum value of three Dickey_fuller p-values. Use this column to decide which zipcode need to take a difference.  
dtest1['max_p'] = dtest1[['Dickey_Fuller_p','Dickey_Fuller_l','Dickey_Fuller_m']].max(axis=1)

#List of zipcode which pass Dickey-Fuller test without taking difference 
diff1_1 = list(dtest1[dtest1.max_p <=0.05].Zipcode)

# List a zipcode which failed Dickey-Fuller test, thus need to take difference
diff2 = list(dtest1[dtest1.max_p > 0.05].Zipcode)


print( f'{ len(diff1_1)} zipcodes passed DF test after taking a first difference.')
print( f'{ len(diff2)} zipcodes failed, so I take a second difference and try again.')



131 zipcodes passed DF test after taking a first difference.
18 zipcodes failed, so I take a second difference and try again.


In [68]:
# for each series check Dickey-Fuller test and report the p-values
dtest2 = pd.DataFrame()
df_p = [] 
df_l = []
df_m = []
Zipcode = []

zips = diff2
for x in zips:
    p_val_1 = adfuller(train_df.diff().diff().dropna().loc[(x, ),]['median_listing_price'])[1] # extract p-value
    p_val_2 = adfuller(train_df.diff().diff().dropna().loc[(x, ),]['active_listing_count'])[1] 
    p_val_3 = adfuller(train_df.diff().diff().dropna().loc[(x, ),]['netinflow'])[1] 
    df_p.append(p_val_1)
    df_l.append(p_val_2)
    df_m.append(p_val_3)
    Zipcode.append(x) 
    
dtest2['Zipcode'] = Zipcode
dtest2['Dickey_Fuller_p'] = df_p
dtest2['Dickey_Fuller_l'] = df_l
dtest2['Dickey_Fuller_m'] = df_m

dtest2.head()

,Zipcode,Dickey_Fuller_p,Dickey_Fuller_l,Dickey_Fuller_m
0,20008,2.775109e-09,3.964328e-05,9.902379e-03
1,20009,6.679153e-09,1.851366e-05,9.308184e-10
2,20011,2.503390e-03,1.365660e-07,1.248371e-07
3,20121,6.552788e-05,5.831161e-04,9.813172e-08
4,20132,1.566768e-04,2.579441e-05,4.245917e-03


In [72]:
# Create a new column of maximum value of three Dickey_fuller p-values. Use this column to decide which zipcode need to take a difference.  
dtest2['max_p'] = dtest2[['Dickey_Fuller_p','Dickey_Fuller_l','Dickey_Fuller_m']].max(axis=1)

#List of zipcode which pass Dickey-Fuller test without taking difference 
diff2_1 = list(dtest2[dtest2.max_p <=0.05].Zipcode)

# List a zipcode which failed Dickey-Fuller test, thus need to take difference
diff3 = list(dtest2[dtest2.max_p > 0.05].Zipcode)

print( f'{ len(diff2_1)} zipcodes passed DF test in 2nd order difference.')
print( f'{ len(diff3)} zipcodes failed and will be dropped from the sample.')


17 zipcodes passed DF test in 2nd order difference.
1 zipcodes failed and will be dropped from the sample.


In below, I drop the zipcodes which failed stationary test after taking 2nd order difference.

In [73]:
# drop from train, test, and alldateset. 
train = train_df.drop(diff3)
test = test_df.drop(diff3)
alldata = all_df.drop(diff3)

In [74]:

print(f'After Granger’s Causality and Dickey Fuller test, my sample includes {alldata.index.unique(level="zipcode").nunique()} zipcodes.' )

print(f'Converage periods are {alldata.index.unique(level="date").min()} to {alldata.index.unique(level="date").max()}.')


After Granger’s Causality and Dickey Fuller test, my sample includes 156 zipcodes.
Converage periods are 2018-01-01 00:00:00 to 2022-04-01 00:00:00.


# 2. Modeling
### Rationality of using VAR (multivariate AR) model 
I chose VAR model for my housing price prediction. It includes three time series; (1)housing price times series (my main focus), (2) number of active listings in an area, and (3) population net inflow in an area. In a competitive market, housing price in each zip code is set by demand and supply in a market. In this model, the number of active listings is a supply variable, and net population inflow captures the size of the demand in an area. As I show in my EDA notebook, these three time series are closely related. 

### Pre-processing to train the model individually for each zip code. 

I train a model for each zip code individually. To streamline the modeling process, I create a list of a data frame. Each data frame is for one zip code. In this way, I can use for loop to run grid search, model validation, and forecasting for all zip codes at once.
I separate a list of a data frame by how many times I took the difference of a time series. Zip codes that passed the DF test without differencing, I stored in (diff0). Zip codes that took the first difference are in (diff1), and zip codes that took two times are in (diff2). This grouping helps me later when I roll back the differenced time series. 

In [75]:
# Define a list of data frames which took 1st difference. 

# Original scale 
train_orig1 = []
test_orig1 = []
test_orig1_1 = []
all_orig1 = []
# Differenced data 
train_diff1 = []
test_diff1 = []
all_diff1 = []

# Store data in the lists

for x in diff1_1:
    train_orig1.append(pd.DataFrame(train.loc[(x, ),]))
    test_orig1.append(pd.DataFrame(test.loc[(x, ),]))
    test_orig1_1.append(pd.DataFrame(test.loc[(x, ),])[1:])
    all_orig1.append(pd.DataFrame(alldata.loc[(x, ),]))
    
    train_diff1.append(pd.DataFrame(train.loc[(x, ),].diff().dropna()))
    test_diff1.append(pd.DataFrame(test.loc[(x, ),].diff().dropna()))
    all_diff1.append(pd.DataFrame(alldata.loc[(x, ),].diff().dropna()))

In [76]:
# Define a list of data frames which takes 2nd difference.

# Original scale 
train_orig2 = []
test_orig2 = []
test_orig2_1 = []
all_orig2 = []
# Differenced data 
train_diff2 = []
test_diff2 = []
all_diff2 = []

# Store data in the lists
for x in diff2_1:
    train_orig2.append(pd.DataFrame(train.loc[(x, ),]))
    test_orig2.append(pd.DataFrame(test.loc[(x, ),]))
    test_orig2_1.append(pd.DataFrame(test.loc[(x, ),])[2:])
    all_orig2.append(pd.DataFrame(alldata.loc[(x, ),]))
    
    train_diff2.append(pd.DataFrame(train.loc[(x, ),].diff().diff().dropna()))
    test_diff2.append(pd.DataFrame(test.loc[(x, ),].diff().diff().dropna()))
    all_diff2.append(pd.DataFrame(alldata.loc[(x, ),].diff().diff().dropna()))

In [77]:
# List of data frame with no differencing 

train_diff0 = []
test_diff0 = []
all_diff0 = []

# Store data in the lists

for x in diff0:
    train_diff0.append(pd.DataFrame(train.loc[(x, ),]))
    test_diff0.append(pd.DataFrame(test.loc[(x, ),]))
    all_diff0.append(pd.DataFrame(alldata.loc[(x, ),]))


In [78]:
# rename the dat frame 
diff2 = diff2_1
diff1 = diff1_1

### Grid search for the order (P) of VAR model 
I run a grid search for each zip code to find the VAR model's best order using train data. For the model selection, I use AIC scores. Because the number of observations in my train dataset is very small for each zip code, I limit my search up to 4 lags.

In [79]:
def grid(data): 
    order = []
    for i in [1,2,3,4]:
        model = VAR(data)
        results = model.fit(i)
        order.append(results.aic)
    order   
    min_value = min(order)
    order = order.index(min_value)+1
    return order

In below, I run grid search function with training data for all zip codes. First, I run for zip codes in diff0 (data without differencing). 

In [80]:
# Run grid search for zip codes in diff0 and store the order in order0   
order0 = []
for df, name in zip(train_diff0, diff0):
    order = grid(df)
    order0.append([name, order])

order0 = pd.DataFrame(order0, columns = ['name','order'])
order0

,name,order
0,20015,2
1,20020,1
2,20151,1
3,22003,2
4,22124,2
5,22152,1
6,22303,1
7,22306,1


The table above report the order with the lowest AIC for each zip code in diff0 list.

I do the same grid search for counties in diff1 list. 

In [81]:
# Grid search for zip codes in diff1  and store the order of the model in order0    
order1 = []
# Use train_diff1 data (one time differenced train data) 
for df, name in zip(train_diff1, diff1):
    order = grid(df)
    order1.append([name, order])

order1 = pd.DataFrame(order1, columns = ['name','order'])
order1.head()

,name,order
0,20001,2
1,20002,1
2,20003,1
3,20005,1
4,20007,4


Finally, I run the grid search for counties in diff2 list. 

In [82]:
# Grid search for zip codes in diff2   and store the order of the model in order0   
order2 = []
# use train_diff2 data (two times differenced data)
for df, name in zip(train_diff2, diff2):
    order = grid(df)
    order2.append([name, order])

order2 = pd.DataFrame(order2, columns = ['name','order'])
order2.head()

,name,order
0,20008,2
1,20009,2
2,20011,3
3,20121,4
4,20132,3


The best orders with the lowest AIC for each zip code is now stored in a data frame named order0, order1, order2.



### Model Validation with test time series

Next, I train the model with selected order from above, and check the model's prediction accuracy using the test time series. For the accuracy score, I use Root Mean Squared Error (RMSE).

In the cell below, I define a function which fits training data on the tuned model and get the prediction for the test period.

In [83]:

# Fit training data on the model with selected order. 
# Make prediction for the test period.   

def fitpredict(train, test, zip_df, order_df): # training data (train_diff0-2), list of zipcodes(diff_df0-2), VAR order(order0-2)   
    # To store the prediction 
    prediction = []
  
    for train_df, name in zip(train, zip_df):
        # 1. Fit the training data into the model
        model = VAR(train_df)
        # get the number of order from the grid serach results
        order = order_df.loc[order_df.name==name, 'order']
        x = int(order)
        model_fitted = model.fit(x)
        # prediction period 
        month = pd.date_range('2021-09-01','2021-12-01', freq='MS')
        
        
        
        # 2. Predict using the train data 
        
        # get the lag order of the fitted model to adjust the length of dataset 
        lag_order = model_fitted.k_ar
        # input data for prediction
        forecast_input = train_df.values[-lag_order:]
        # Prediction for the test period (this case 2021-09-01 to 2021-12-01)
        i = zip_df.index(name)
        fc = model_fitted.forecast(y=forecast_input, steps=4) # steps = the test period #steps=len(test[i])
        df_forecast = pd.DataFrame(fc, columns = ['median_listing_price','active_listing_count', 'netinflow'], index=month) #index=test[i].index
        prediction.append(df_forecast)
        

    return prediction

in below, I apply the fitpredict function for all zipcode. 

In [84]:
# Input the zipcodes, train data, and VAR order into fitprediction dunction. 
# 0, 1, 2 indicates number of difference of time series, and zipcodes are stored by the number of difference.  

predict0 = fitpredict(train_diff0, test_diff0, diff0, order0)
predict1 = fitpredict(train_diff1, test_diff1, diff1, order1)
predict2 = fitpredict(train_diff2, test_diff2, diff2, order2)

### Rolling back differenced time series.
Before evaluating the predicted values, I need to bring the differenced data back up to its original scale. In the following cels, I create a function to roll back the differenced data. For counties in diff1, I differenced the data one time, so the prediction is one time difference. To roll back to the original scale, I sum up all differences and add it back to the last observed data of the training set. The below is the function to roll back for the differenced data.

The function below rolls back the first order differencing to the original scale.  

In [85]:
# This function roll back the first order differencing to get the original scale  

def invert_diff1(df_train, df_forecast):
    """Revert back the first differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:    
        # create cumusum for forcasting (differenced data) 
        df_fc[str(col)+'_cumsum'] = df_fc[str(col)].cumsum()
        # add a column of the last observed data from the training. Here, this training data should be stored in the original scale. 
        df_fc[str(col)+'_last'] = df_train[col].iloc[-1] 
        # add the acumulative change to the last observed data in original scale.
        df_fc[str(col)+'_o'] = df_fc[str(col)+'_last'] + df_fc[str(col)+'_cumsum'] 
    df_fc_o = df_fc[['median_listing_price_o', 'active_listing_count_o', 'netinflow_o']]
    # rename the columns of df_fc_o to match with the original 
    df_fc_o.rename(columns={'median_listing_price_o':'median_listing_price', 'active_listing_count_o':'active_listing_count', 
                   'netinflow_o': 'netinflow'}, inplace=True)

    return df_fc_o

I apply the roll-back function defined above to one-time differenced time series. 

In [86]:
predict1_rolled = []
for df_train, df_pre, name in zip(train_orig1, predict1, diff1):
    # apply invert difference function. 
    df_fc = invert_diff1(df_train, df_pre)
    predict1_rolled.append(df_fc)



The next function rolls back the second order differencing to get the original scale.  

In [87]:
# This function roll back the second order differencing to get the original scale  

def invert_diff2(df_train, df_forecast):
    """Revert back the second order differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:    
        # create cumulative sum of forcasting (= accumulative 2nd order change)  
        df_fc[str(col)+'_cumsum2d'] = df_fc[str(col)].cumsum()
        # add a column of the last observed data from the training. Here, this training data should be stored in the original scale. 
        df_fc[str(col)+'_last'] = df_train[col].iloc[-1] 
        # add a column of the second last observed data from the training. 
        df_fc[str(col)+'_2last'] = df_train[col].iloc[-2] 
        # change of the second last observed and the last observed 
        df_fc[str(col)+'_change'] =  df_fc[str(col)+'_last'] - df_fc[str(col)+'_2last']

        # add the acumulative 2nd change +  the last observed change + the last observed original data   
        df_fc[str(col)+'_o'] = df_fc[str(col)+'_last'] + df_fc[str(col)+'_change']+ df_fc[str(col)+'_cumsum2d'] 
    
    df_fc_o = df_fc[['median_listing_price_o', 'active_listing_count_o', 'netinflow_o']]
    # rename the columns of df_fc_o to match with the original 
    df_fc_o.rename(columns={'median_listing_price_o':'median_listing_price', 'active_listing_count_o':'active_listing_count', 
                   'netinflow_o': 'netinflow'}, inplace=True)
    return df_fc_o



In [88]:
predict2_rolled = []
for df_train, df_pre, name in zip(train_orig2, predict2, diff2):
    # apply invert difference function. 
    df_fc = invert_diff2(df_train, df_pre)
    predict2_rolled.append(df_fc)



To calculate RMSE, I adjust the data frame for the first or second differenced time series. Because I took the difference after the train test split, the test data for these time series are fewer than the prediction. So I drop the corresponding rows from the prediction data.


In [89]:
predict1_rolled_1 = []

for df, name in zip(predict1_rolled, diff1):
    df1 = df[1:]
    predict1_rolled_1.append(df1)

predict2_rolled_1 = []

for df, name in zip(predict2_rolled, diff2):
    df2 = df[2:]
    predict2_rolled_1.append(df2)


Using the rolled back prediction, I calculate RMSE. 

#### RMSE Function

In [90]:
def rmse(test, predict, zip_df):
    # To store RMSE for each variables.  
    summary_rmse = pd.DataFrame()
    RMSE1=[]
    RMSE2=[]
    RMSE3=[]
    Zipcode=[]
        
    for df, name in zip(predict, zip_df):
        
        predict1 = df['median_listing_price']
        predict2 = df['active_listing_count']
        predict3 = df['netinflow']
        
        i = zip_df.index(name)
        test1 = test[i]['median_listing_price']
        test2 = test[i]['active_listing_count']
        test3 = test[i]['netinflow']
        
        rmse1 = np.sqrt(mean_squared_error(test1, predict1))
        rmse2 = np.sqrt(mean_squared_error(test2, predict2))
        rmse3 = np.sqrt(mean_squared_error(test3, predict3))
        
        Zipcode.append(name)
        RMSE1.append(rmse1)
        RMSE2.append(rmse2)
        RMSE3.append(rmse3)
        
    summary_rmse['Zipcode'] = Zipcode
    summary_rmse['RMSE_price'] = RMSE1
    summary_rmse['RMSE_listing'] = RMSE2
    summary_rmse['RMSE_inflow'] = RMSE3
    
    return summary_rmse


I apply RMSE function for each zip code. RMSE for all zip codes are stored in one table as below. 

In [91]:
# Calcuate RMSE by using rmse function 
rmse_df0 = rmse(test_diff0, predict0, diff0) 
rmse_df1 = rmse(test_orig1_1, predict1_rolled_1, diff1)  #test_orig1, 
rmse_df2 = rmse(test_orig2_1, predict2_rolled_1, diff2)
# Create one large dataframe wchich store the all RMSE 
rmse_df = rmse_df0.append(rmse_df1).append(rmse_df2)
rmse_df

,Zipcode,RMSE_price,RMSE_listing,RMSE_inflow
0,20015,268848.075409,6.343578,15.516029
1,20020,37489.428987,14.554757,39.955205
2,20151,91774.131989,12.019732,21.361424
3,22003,73768.134405,13.005209,17.689678
4,22124,277594.794481,20.312284,24.079081
...,...,...,...,...
12,21704,169356.826272,11.547151,64.881706
13,21774,29952.477286,18.170015,75.953818
14,22066,292142.946693,12.957499,33.774975
15,22180,172843.294952,4.303384,67.025676


#### Comparison with Naive model's RMSE
To evaluate VAR model,  I compare with RMSE of naive model. Naive model for the time series is shifting the time series by one period. So, the next week's revenue change is equal to today's revenue change. In below, I calcuate RMSE of naive model for all zip codes.

In [92]:
# Calculate a prediction by naive model 

def rmse_naive(data, zip_df):
    
    # to store RMSE for all zipcodes 
    rmse_df = pd.DataFrame()
    rmse_naive = []
    Zipcode = []
    
    for df, name in zip(data, zip_df):
        
        # Get housing price 
        price = df['median_listing_price']
        # Naive model prediction  
        naive = price.shift(1)
        # Calculate RMSE 
        rmse = np.sqrt(mean_squared_error(price[1:], naive.dropna()))
        # store 
        rmse_naive.append(rmse)
        Zipcode.append(name)
    # store results in data frame 
    rmse_df['RMSE_naive'] = rmse_naive
    rmse_df['Zipcode'] = Zipcode
    
    return rmse_df

Using the anove function, I calculate the naive model RMSE for all zip codes. 

In [93]:
# Calcuate naive model RMSE 
rmse_naive0 = rmse_naive(all_diff0, diff0) 
rmse_naive1 = rmse_naive(all_diff1, diff1)  #test_orig1, 
rmse_naive2 = rmse_naive(all_diff2, diff2)
# Create one large dataframe wchich store the all RMSE 
rmse_naive_df = rmse_naive0.append(rmse_naive1).append(rmse_naive2)
rmse_naive_df.head()


,RMSE_naive,Zipcode
0,227335.444792,20015
1,23779.996449,20020
2,54357.833532,20151
3,41833.691498,22003
4,186632.169037,22124


In [94]:
# Comparison 
# Merge to VAR's rmse table 
rmse_df_1 = rmse_df.merge(rmse_naive_df, on='Zipcode', how='left')
# Compare 
rmse_df_1.describe()

,RMSE_price,RMSE_listing,RMSE_inflow,RMSE_naive
count,1.560000e+02,156.000000,156.000000,156.000000
mean,9.444711e+04,10.368096,32.686321,88836.015021
std,1.606751e+05,9.513676,24.503454,103566.573825
min,2.265224e+03,0.561967,3.799956,14586.772786
25%,2.918630e+04,4.429202,17.632170,36997.528137
50%,4.904863e+04,7.373662,26.006453,50836.962086
75%,9.393838e+04,12.441012,38.516547,92681.848929
max,1.610229e+06,68.779270,184.232534,687712.511380


RMSE of housing price by VAR model widely varies across zipcodes (from 3641 to 545165). Median RMSE is 32300. Given the majority of housing pice are around 600,000, this prediction is not bad. My model's prediction will be off the real value by 6%.  

Comparison with narive model RMSE shows that mean, median, min, max values of RMSE for the VA model is significantly lower than naive model's RMSE. So, my VAR model improved the prediction accuracy.  



# 4  Forecasting

I forecast the housing price for 2022 for all zip codes. First, I fit the model into an entire sample. Using that model, I make an out-of-sample prediction.

In the function below, I fit the model on the complete dataset and get a forecast for 2022 for zip code. The following function is almost identical to the earlier fitprediction function with minor changes.

In [95]:

def forecast(data, zip_df, order_df): # complete dataset (all_diff0-2), list of zipcodes(diff_df0-2), VAR order(order0-2)   
    """
    Input: complete train data, list of county ids, and exogenous data. 
    Output: predictions
    
    """
    
    # To store 12 months forecast 
    forecast12 = []

  
    for df, zipcode in zip(data, zip_df):
        
        # 1. Fit the model on the complete dataset
        model = VAR(df)
        # get the number of order from the grid serach results
        order = order_df.loc[order_df.name==zipcode, 'order']
        x = int(order)
        model_fitted = model.fit(x)
        
        
        # 2. Get forecast for the next 12 month  
        
        # get the lag order of the fitted model 
        lag_order = model_fitted.k_ar
        # input data for forecasting
        forecast_input = df.values[-lag_order:]
        # forecasting months 
        month = pd.date_range('2022-01-01','2022-12-01', freq='MS')
        
        # Get forecast for 2022-01-01 to 2022-12-01)
        fc = model_fitted.forecast(y=forecast_input, steps=12) # steps = the test period 
        df_forecast = pd.DataFrame(fc, columns = ['median_listing_price','active_listing_count', 'netinflow'], index=month) #, index=month
                
        forecast12.append(df_forecast)

        

    return forecast12



Apply the function and get prediction.

In [96]:
# Input the zipcodes, train data, and VAR order into fitprediction dunction. 
# 0, 1, 2 indicates number of difference of time series, and zipcodes are stored by the number of difference.  

forecast0 = forecast(all_diff0, diff0, order0)
forecast1 = forecast(all_diff1, diff1, order1)
forecast2 = forecast(all_diff2, diff2, order2)

#### Rolling back differenced time series.
Using invert_diff function created earlier, I roll back the differenced time series (diff1 and diff2) to its original scale.

First, I apply the invert_diff1 function to roll back the first order differencing to get the original scale 

In [97]:
forecast1_rolled = []
for df_train, df_pre, name in zip(train_orig1, forecast1, diff1):
    # apply invert difference function. 
    df_fc = invert_diff1(df_train, df_pre)
    forecast1_rolled.append(df_fc)



Next, I apply the invert_diff2 function to roll back the second order differencing to get the original scale 

In [98]:
forecast2_rolled = []
for df_train, df_pre, name in zip(train_orig2, forecast2, diff2):
    # apply invert difference function. 
    df_fc = invert_diff2(df_train, df_pre)
    forecast2_rolled.append(df_fc)


### Store forecast in one data frame

Forecasted values are stored in a separate data by number of differencing. In the cell below, I create a one large data frame which sored forecasted value in one data frame. 

In [99]:
# Store forecasted values in list of data frame format. 
forecast0_df = []
for df_pre, name in zip(forecast0, diff0):
    df_pre['zipcode']=name
    forecast0_df.append(df_pre)

forecast1_df = []
for df_pre, name in zip(forecast1_rolled, diff1):
    df_pre['zipcode']=name
    forecast1_df.append(df_pre)
    
forecast2_df = []
for df_pre, name in zip(forecast2_rolled, diff2):
    df_pre['zipcode']=name
    forecast2_df.append(df_pre)

# merge a list of DataFrames into a single DataFrame
fc_0 = pd.concat(forecast0_df)
fc_1 = pd.concat(forecast1_df)
fc_2 = pd.concat(forecast2_df)

# append fc_0, fc_1, fc_2
fc = fc_0.append(fc_1).append(fc_2)

# merge forecast dataframe and observed data 
DC1 = DC.reset_index()
fc1 = fc.reset_index()
fc1.rename(columns={'index':'date'}, inplace=True )
dcforecast = DC1.merge(fc1, on = ['date','zipcode'], how='outer')

# forecast and observed price are currently in the different column. Now append forecast to the historic time series  
dcforecast.median_listing_price_x.fillna(dcforecast.median_listing_price_y, inplace=True)
dcforecast.netinflow_x.fillna(dcforecast.netinflow_y, inplace=True)
dcforecast.active_listing_count_x.fillna(dcforecast.active_listing_count_y, inplace=True)

# Drop forecast columns and rename columns. 
dcforecast.drop(['median_listing_price_y', 'active_listing_count_y', 'netinflow_y'], axis=1, inplace=True)
dcforecast.rename(columns={'median_listing_price_x': 'median_listing_price', 
                          'active_listing_count_x':'active_listing_count', 'netinflow_x':'netinflow' }, inplace=True)



Finally, I calculate yearly changes of the forecast and save it in the csvfile. 

In [100]:
# Calcualte the yearly changes

dcforecast['change_price'] = dcforecast.groupby(['zipcode'])['median_listing_price'].pct_change(12)
dcforecast['change_listing'] = dcforecast.groupby(['zipcode'])['active_listing_count'].pct_change(12)
dcforecast['change_inflow'] = dcforecast.groupby(['zipcode'])['netinflow'].pct_change(12)


# # save in csv 
fc.to_csv('Data/dcforecast12.csv', index=False)
DC.to_csv('Data/dchistoric.csv', index=False)
dcforecast.to_csv('Data/dcfull.csv', index=False)


In [101]:
# Calcualte the yearly changes

# Set Multiinde x with daye and zip code
df_final = dcforecast.set_index(['zipcode', 'date']).sort_index() 
# create a data frame which shifted by 12 month. 
df_shifted = df_final.shift(12).head(20)


# merge shifted data and original data
df_shifted.reset_index(inplace=True)
df_final.reset_index(inplace=True)
df_save = df_final.merge(df_shifted, on=['zipcode', 'date'], how='left')

# calculate the year on year changes. 
df_save['price_change'] = df_save.median_listing_price_x - df_save.median_listing_price_y
df_save['listing_change'] = df_save.active_listing_count_x - df_save.active_listing_count_y
df_save['inflow_change'] = df_save.netinflow_x - df_save.netinflow_y

# Drop unnecessary columns 
drop=['median_listing_price_y', 'active_listing_count_y', 'netinflow_y', ]
df_save.drop(drop, axis=1, inplace=True)
df_save.rename(columns={'median_listing_price_x':'median_listing_price', 'active_listing_count_x':'active_listing_count',
       'netinflow_x':'netinflow'}, inplace=True)



In [102]:
df_save.to_csv('Data/housepricefc.csv')